# select the analysis participant base for gambling engagement (it meatters in normalization/outlier deletion)

In [1]:
library(dplyr)
# year 2016
data_2016 = read.csv('../output/data_cleaned/HarmSurvey2016.csv', row =1)
data_gambled = data_2016 %>% select(subjectkey, starts_with('freq')) %>% tibble::column_to_rownames('subjectkey')
# 8 is an un-used code 
data_gambled[is.na(data_gambled)] = 8
# 1-7 from 'daily' to 'not at all'
data_gambled$max_activity = apply(data_gambled, 1, min)
data_gambled$I_gambled = ifelse(data_gambled$max_activity < 7, 1, 0)

data_gambled_2016 = data_gambled %>% filter(I_gambled == 1) %>% select(max_activity) %>% tibble::rownames_to_column('subjectkey')
dim(data_gambled_2016)
head(data_gambled_2016,2)

# year 2017
data_2017 = read.csv('../output/data_cleaned/HarmSurvey2017.csv', row =1)
data_gambled = data_2017 %>% select(subjectkey, starts_with('freq')) %>% tibble::column_to_rownames('subjectkey')
# 8 is an un-used code 
data_gambled[is.na(data_gambled)] = 8
# 1-7 from 'daily' to 'not at all'
data_gambled$max_activity = apply(data_gambled, 1, min)
data_gambled$I_gambled = ifelse(data_gambled$max_activity < 7, 1, 0)

data_gambled_2017 = data_gambled %>% filter(I_gambled == 1) %>% select(max_activity) %>% tibble::rownames_to_column('subjectkey')
dim(data_gambled_2017)

Warning message:
"package 'dplyr' was built under R version 4.2.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[1] 5766    2

,subjectkey,max_activity
,<chr>,<dbl>
1,1,4
2,2,2


[1] 2161    2

## gambling frequency

In [2]:
#########2016

activity_numeric = data_2016 %>% select(subjectkey,starts_with('freq_')) %>% tidyr::drop_na()

library(tidyverse)
# reverse coding, original: daily 1; to not at all: 7; cannot say 9
rownames(activity_numeric) <- NULL
activity_numeric_new = activity_numeric %>% tibble::column_to_rownames('subjectkey')
freq_df_r = 8 - activity_numeric_new

freq_df_r[freq_df_r == -1] = NA
freq_df = na.omit(freq_df_r)

data_pca = freq_df %>% select(-freq_Lotto)
pca_model = prcomp(data_pca, scale = TRUE)

freq_df_pca = as.data.frame(pca_model$x) %>% 
            select(PC1) %>%
            tibble::rownames_to_column('subjectkey') %>%
            rename(sum_freq_all_No_lottery = PC1)

head(freq_df_pca)
dim(freq_df_pca)

write.csv(freq_df_pca, '../output/data_cleaned/freq_df_summary_2016_PCA.csv')

#########2017
library(dplyr)

activity_numeric = data_2017 %>% select(subjectkey,starts_with('freq_')) %>% tidyr::drop_na()

library(tidyverse)
# reverse coding, original: daily 1; to not at all: 7; cannot say 9
rownames(activity_numeric) <- NULL
activity_numeric_new = activity_numeric %>% tibble::column_to_rownames('subjectkey')
freq_df_r = 8 - activity_numeric_new

freq_df_r[freq_df_r == -1] = NA
freq_df = na.omit(freq_df_r)

data_pca = freq_df %>% select(-freq_Lotto)
pca_model = prcomp(data_pca, scale = TRUE)

freq_df_pca = as.data.frame(pca_model$x) %>% 
            select(PC1) %>%
            tibble::rownames_to_column('subjectkey') %>%
            rename(sum_freq_all_No_lottery = PC1)

head(freq_df_pca)
dim(freq_df_pca)

write.csv(freq_df_pca, '../output/data_cleaned/freq_df_summary_2017_PCA.csv')

Warning message:
"package 'tidyverse' was built under R version 4.2.3"
Warning message:
"package 'ggplot2' was built under R version 4.2.3"
Warning message:
"package 'tibble' was built under R version 4.2.3"
Warning message:
"package 'readr' was built under R version 4.2.3"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.1     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


,subjectkey,sum_freq_all_No_lottery
,<chr>,<dbl>
1,1,-0.6282489
2,2,-0.3094462
3,3,0.3171101
4,4,-0.3607761
5,6,-1.0201452
6,7,-0.6453903


[1] 6562    2

,subjectkey,sum_freq_all_No_lottery
,<chr>,<dbl>
1,1,-0.7863862
2,3,-0.7168968
3,5,1.2139401
4,6,-0.8655303
5,7,-0.8655303
6,8,-0.4137983


[1] 2400    2

## gambling expenditures

In [3]:
######2016
data_2016_recode = read.csv('../output/data_cleaned/HarmSurvey2016_recode.csv',row = 1)

library(dplyr)

# select who gambled last year
who_gambled_2016 = data_gambled_2016$subjectkey 

data_expense = data_2016_recode %>% filter(subjectkey %in% who_gambled_2016) %>%
            select(subjectkey,starts_with('spent')) %>% 
            mutate(spent_per_w = as.numeric(spent_per_w),
                   spent_per_m = as.numeric(spent_per_m),
                   spent_the_y = as.numeric(spent_the_y)
                  ) %>%
                mutate(annual_expenses = 
                       ifelse(!is.na(spent_the_y), spent_the_y, 
                              ifelse(!is.na(spent_per_m), spent_per_m * 12, spent_per_w * 52))) %>%
                select(subjectkey, annual_expenses) %>% tidyr::drop_na()

cat('all gamblers without missing data: ', nrow(data_expense), '\n')
# clean 0 and 100000; and replace outliers higher than three sd with upper limit
spent_drop_negative = data_expense %>% filter(annual_expenses > 0)
cat('non-positive expenditures : ', nrow(data_expense) - nrow(spent_drop_negative), '\n')

spent_drop_toomuch = spent_drop_negative %>% filter(annual_expenses < 100000)
cat('exceeding 100,000 euros: ', nrow(spent_drop_negative) - nrow(spent_drop_toomuch), '\n')

uppper_limit = mean(spent_drop_toomuch$annual_expenses) + 3* sd(spent_drop_toomuch$annual_expenses)
spent_drop_toomuch$annual_expenses[spent_drop_toomuch$annual_expenses > uppper_limit] = uppper_limit
spent_2016 = spent_drop_toomuch
head(spent_2016,2)
dim(spent_2016)
write.csv(spent_2016,'../output/data_cleaned/wmy_expenses_df_2016.csv')

# -------------------2017---------------------------------
data_2017_recode = read.csv('../output/data_cleaned/HarmSurvey2017_recode.csv',row = 1)

library(dplyr)

# select who gambled last year
who_gambled_2017 = data_gambled_2017$subjectkey 

data_expense = data_2017_recode %>% filter(subjectkey %in% who_gambled_2017) %>%
            select(subjectkey,starts_with('spent')) %>% 
            mutate(spent_per_w = as.numeric(spent_per_w),
                   spent_per_m = as.numeric(spent_per_m),
                   spent_the_y = as.numeric(spent_the_y)
                  ) %>%
                mutate(annual_expenses = 
                       ifelse(!is.na(spent_the_y), spent_the_y, 
                              ifelse(!is.na(spent_per_m), spent_per_m * 12, spent_per_w * 52))) %>%
                select(subjectkey, annual_expenses) %>% tidyr::drop_na()

cat('all gamblers without missing data: ', nrow(data_expense), '\n')
# clean 0 and 100000; and replace outliers higher than three sd with upper limit
spent_drop_negative = data_expense %>% filter(annual_expenses > 0)
cat('non-positive expenditures : ', nrow(data_expense) - nrow(spent_drop_negative), '\n')

spent_drop_toomuch = spent_drop_negative %>% filter(annual_expenses < 100000)
cat('exceeding 100,000 euros: ', nrow(spent_drop_negative) - nrow(spent_drop_toomuch), '\n')

uppper_limit = mean(spent_drop_toomuch$annual_expenses) + 3* sd(spent_drop_toomuch$annual_expenses)
spent_drop_toomuch$annual_expenses[spent_drop_toomuch$annual_expenses > uppper_limit] = uppper_limit
spent_2017 = spent_drop_toomuch
head(spent_2017,2)
dim(spent_2017)
write.csv(spent_2017,'../output/data_cleaned/wmy_expenses_df_2017.csv')

Warning message:
"There were 3 warnings in `mutate()`.
The first warning was:
ℹ In argument: `spent_per_w = as.numeric(spent_per_w)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 2 remaining warnings."


all gamblers without missing data:  4509 
non-positive expenditures :  210 
exceeding 100,000 euros:  2 


,subjectkey,annual_expenses
,<int>,<dbl>
1,1,100
2,2,780


[1] 4297    2

Warning message:
"There were 3 warnings in `mutate()`.
The first warning was:
ℹ In argument: `spent_per_w = as.numeric(spent_per_w)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 2 remaining warnings."


all gamblers without missing data:  1713 
non-positive expenditures :  23 
exceeding 100,000 euros:  1 


,subjectkey,annual_expenses
,<int>,<dbl>
1,1,150
2,2,1040


[1] 1689    2

## gambling diversity

In [4]:
########2016##############
# dichotomize vlues
activity_numeric = data_2016 %>% select(subjectkey,starts_with('freq_')) %>% tidyr::drop_na()
rownames(activity_numeric) <- NULL
width_raw = activity_numeric %>% tibble::column_to_rownames('subjectkey')

width_raw[width_raw == 9] = NA
width_raw[width_raw == 7] = 0
width_raw[width_raw > 0] = 1

# summerize
width_df_summary = width_raw %>% mutate(diverity = rowSums(across(where(is.numeric))))%>%
                                    mutate(diverity_No_lottery = diverity - freq_Lotto) %>%
                                    tibble::rownames_to_column('subjectkey') %>% tidyr::drop_na()

head(width_df_summary,2)
dim(width_df_summary)
write.csv(width_df_summary, '../output/data_cleaned/width_df_summary_2016.csv')

#########2017###########
activity_numeric = data_2017 %>% select(subjectkey,starts_with('freq_')) %>% tidyr::drop_na()
rownames(activity_numeric) <- NULL
width_raw = activity_numeric %>% tibble::column_to_rownames('subjectkey')

width_raw[width_raw == 9] = NA
width_raw[width_raw == 7] = 0
width_raw[width_raw > 0] = 1

# summerize
width_df_summary = width_raw %>% mutate(diverity = rowSums(across(where(is.numeric))))%>%
                                    mutate(diverity_No_lottery = diverity - freq_Lotto) %>%
                                    tibble::rownames_to_column('subjectkey') %>% tidyr::drop_na()

dim(width_df_summary)
write.csv(width_df_summary, '../output/data_cleaned/width_df_summary_2017.csv')

,subjectkey,freq_Lotto,freq_OthLotA,freq_OthLotB,freq_Scratch,freq_VSbet,freq_VHorse,freq_Casino,freq_Slots,freq_Table,⋯,freq_FHorse,freq_Private,freq_Cruise,freq_PPaf,freq_OPaf,freq_PFor,freq_OFor,freq_Other,diverity,diverity_No_lottery
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0,1,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,3,2
2,2,1,0,1,1,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,3,2


[1] 6562   22

[1] 2400   22